In [6]:
from pprint import pprint

import sklearn.datasets
import sklearn.metrics

import autosklearn.regression

In [1]:
from pprint import pprint

import sklearn.metrics

import autosklearn.regression
from settings import INPUT_FILE, CORR_GROUP, OUTPUT_FILE
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging

logging.basicConfig(format='%(asctime)s %(message)s', filename='automl.log', level=logging.DEBUG)
logging.info('Started training')

def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)


def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

if __name__ == '__main__':
    df = pd.read_csv(INPUT_FILE, index_col='ts')
    #df = df.drop('Unnamed: 0', 1)
    df.index = pd.to_datetime(df.index)

    df_2 = df.loc[:,np.invert(unique_cols(df))]
    # Average window
    # df_2 = df.groupby(np.arange(len(df))//60).mean()

    scaler = MinMaxScaler()
    d = scaler.fit_transform(df_2)
    scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)
    results = []
    auto_ml_desc = {}
    k = 'H_TDH_U_L2_N'
    logging.info(k + " started training")
    values = create_supervised_dataset(scaled_df, k, CORR_GROUP[k], n_in=15, n_out=1)
    len_values = values.shape[0]
    # split into train and test sets 
    n_train_seconds = int(0.7*len_values) #70% dos valores
    n_cv_seconds =  int(1*len_values) #20% dos valores
    train = values[:n_train_seconds, :]
    cv = values[n_train_seconds:n_cv_seconds, :]

    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1:]
    test_X, test_y = cv[:, :-1], cv[:, -1:]
    automl = autosklearn.regression.AutoSklearnRegressor(
        time_left_for_this_task=7200,
        per_run_time_limit=120,
        tmp_folder='./tmp/autosklearn_regression_'+k+'_influx_tmp_4',
    )
    try:
        automl.fit(train_X, train_y, dataset_name=k)
        logging.info(k+' model predicting')
        automl.performance_over_time_.to_csv('results/history_'+k+'.csv')
        test_predictions = automl.predict(test_X)
        results.append(sklearn.metrics.mean_squared_error(test_y, test_predictions, squared=False))
    except Exception as e:
        logging.exception(e)
        results.append(1)
    print(results)

/home/atnoguser/git/Electricity-Forecasting/venv/lib/python3.8/site-packages/autosklearn/metalearning/metalearning/meta_base.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.metafeatures = self.metafeatures.append(metafeatures)
/home/atnoguser/git/Electricity-Forecasting/venv/lib/python3.8/site-packages/autosklearn/metalearning/metalearning/meta_base.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.algorithm_runs[metric].append(runs)


[0.03414104364767699]


In [4]:
for i in auto_ml_desc:
    auto_ml_desc[i].to_csv('results/history_'+i+'.csv')

In [5]:
import json
  
with open("myfile.txt", 'w') as f: 
    for key, value in auto_ml_desc.items(): 
        f.write('%s:%s\n' % (key, value))

In [22]:
!pip3 install --force-reinstall autokeras


  Using cached autokeras-1.0.19-py3-none-any.whl (162 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
     |████████████████████████████████| 11.7 MB 3.6 MB/s eta 0:00:01    |██████████████████▍             | 6.7 MB 3.6 MB/s eta 0:00:02
  Using cached tensorflow-2.9.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
  Using cached keras_tuner-1.1.2-py3-none-any.whl (133 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached ipython-8.3.0-py3-none-any.whl (750 kB)
  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)
     |████████████████████████████████| 16.9 MB 27.4 MB/s eta 0:00:01    |███████▉                        | 4.1 MB 27.4 MB/s eta 0:00:01��████████████████             | 10.0 MB 27.4 MB/s eta 0:00:01
  Using cached tensorboard-2.9.0-py3-none-any.whl (5.8 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     |████████████████████████████████| 2.4 MB 32.1 MB/s eta 0:00:01
  Using cached google_pa

In [29]:
import autokeras
print(dir(autokeras))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [5]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

from settings import INPUT_FILE, CORR_GROUP, OUTPUT_FILE
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging

logging.basicConfig(format='%(asctime)s %(message)s', filename='autokeras.log', level=logging.DEBUG)
logging.info('Started training')

def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)

def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)

df_2 = df.loc[:,np.invert(unique_cols(df))]
# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()

scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)
results = []
train_split = int(0.07*scaled_df.shape[0])
val_split = int(0.09*scaled_df.shape[0])
predict_from = 1
predict_until = 1
lookback = 15

for k in CORR_GROUP:
    data_train = scaled_df[:train_split]
    data_cv = scaled_df[train_split:val_split]
    
    data_x = data_train[CORR_GROUP[k]]
    data_y = data_train[k]

    data_x_val = data_cv[CORR_GROUP[k]]
    data_y_val = data_cv[k]

    data_test = create_supervised_dataset(scaled_df, k, CORR_GROUP[k], n_in=15, n_out=1)[val_split:int(0.1*scaled_df.shape[0])]
    data_x_test, data_y_test = data_test[:, :-1], data_test[:, -1]
    data_x_test = data_x_test.reshape(-1, 15, len(CORR_GROUP[k]))
    
    print(data_x.shape, data_y.shape)
    print(data_x_val.shape, data_y_val.shape)
    print(data_x_test.shape, data_y_test.shape)

    clf = ak.TimeseriesForecaster(
        lookback=lookback,
        predict_from=predict_from,
        predict_until=predict_until,
        max_trials=3,
        project_name=f'{k}_forecaster',
        objective="mean_squared_error",
    )

    # Train the TimeSeriesForecaster with train data
    clf.fit(
        x=data_x,
        y=data_y,
        validation_data=(data_x_val, data_y_val),
        batch_size=32,
        epochs=10,
        verbose=False
    )
    model = clf.export_model()
    # Evaluate the best model with testing data.
    print(model.evaluate(data_x_test, data_y_test)[0])
    

(2768, 19) (2768,)
(791, 19) (791,)
(396, 15, 19) (396,)
INFO:tensorflow:Reloading Oracle from existing project ./P_SUM_forecaster/oracle.json


KeyboardInterrupt: 

In [16]:
def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

In [28]:
val_split = int(0.09*scaled_df.shape[0])
data_test = scaled_df[val_split:int(0.1*scaled_df.shape[0])]
values = create_supervised_dataset(scaled_df, 'P_SUM', CORR_GROUP['P_SUM'], n_in=15, n_out=1)[val_split:int(0.1*scaled_df.shape[0])]
data_x_test, data_y_test = values[:, :-1], values[:, -1]
data_x_test = data_x_test.reshape(-1, 15, 19)
print(data_x_test.shape)
print(data_y_test.shape)
model = clf.export_model()
model.evaluate(data_x_test, data_y_test)

(396, 15, 19)
(396,)
13/13 [==============================] - 2s 8ms/step - loss: 0.0096 - mean_squared_error: 0.0096


[0.00960538350045681, 0.00960538350045681]

In [29]:
print(model.metrics_names)

['loss', 'mean_squared_error']


Results from 1hour of training: [0.06908108540832145, 0.02820249277020434, 0.05765261241919069, 0.034627797684058464, 0.04013057183835941, 0.02842411915671778, 0.04400331222349673, 0.025787262329089365, 0.16661010825624262, 0.03407722280903902]

In [ ]:
def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)


def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values


In [9]:
df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)

df_2 = df.loc[:,np.invert(unique_cols(df))]
# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()

scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)
scaled_df['RealEc_L1'].tail(1000)

ts
2022-04-13 16:04:00    0.99989
2022-04-13 16:05:00    0.99989
2022-04-13 16:06:00    0.99989
2022-04-13 16:07:00    0.99989
2022-04-13 16:08:00    0.99989
                        ...   
2022-04-14 08:42:00    1.00000
2022-04-14 08:43:00    1.00000
2022-04-14 08:44:00    1.00000
2022-04-14 08:45:00    1.00000
2022-04-14 08:46:00    1.00000
Name: RealEc_L1, Length: 1000, dtype: float64

In [4]:
df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)

df_2 = df.loc[:,np.invert(unique_cols(df))]
# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()

scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)
results = []

k = 'ReacEc_L1'

print(k + " started training")
values = create_supervised_dataset(scaled_df, k, CORR_GROUP[k], n_in=15, n_out=1)
len_values = values.shape[0]

# split into train and test sets 
n_train_seconds = int(0.7*len_values) #70% dos valores
n_cv_seconds =  int(1*len_values) #20% dos valores
train = values[:n_train_seconds, :]
cv = values[n_train_seconds:n_cv_seconds, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1:]
test_X, test_y = cv[:, :-1], cv[:, -1:]

print(train_y)

automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='./tmp/autosklearn_regression_'+k+'_tmp_3',
)
automl.fit(train_X, train_y, dataset_name=k)
test_predictions = automl.predict(test_X)

ReacEc_L1 started training
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


ValueError: AutoMLRegressor does not support task binary

In [2]:
def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)


def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

In [7]:
from pprint import pprint

import sklearn.metrics

import autosklearn.regression
from settings import INPUT_FILE, CORR_GROUP, OUTPUT_FILE
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging


In [15]:
from settings import INPUT_FILE
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)

df_2 = df.loc[:,np.invert(unique_cols(df))]

# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()

scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)

In [4]:
from settings import CORR_GROUP

for k in CORR_GROUP:
    values = create_supervised_dataset(scaled_df, k, CORR_GROUP[k], n_in=15, n_out=1)
    len_values = values.shape[0]
    # split into train and test sets 
    n_train_seconds = int(0.7*len_values) #70% dos valores
    n_cv_seconds =  int(1*len_values) #20% dos valores
    train = values[:n_train_seconds, :]
    cv = values[n_train_seconds:n_cv_seconds, :]

    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1:]
    test_X, test_y = cv[:, :-1], cv[:, -1:]
    automl = autosklearn.regression.AutoSklearnRegressor(
        time_left_for_this_task=120,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
    )

    automl.fit(train_X, train_y, dataset_name='diabetes')
    break



In [5]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='./tmp/autosklearn_regression_example_tmp',
)
automl.fit(X_train, y_train, dataset_name='diabetes')

/home/atnoguser/git/Electricity-Forecasting/venv/lib/python3.8/site-packages/autosklearn/metalearning/metalearning/meta_base.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.metafeatures = self.metafeatures.append(metafeatures)
/home/atnoguser/git/Electricity-Forecasting/venv/lib/python3.8/site-packages/autosklearn/metalearning/metalearning/meta_base.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.algorithm_runs[metric].append(runs)


AutoSklearnRegressor(per_run_time_limit=30, time_left_for_this_task=120,
                     tmp_folder='./tmp/autosklearn_regression_example_tmp')

In [6]:
print(automl.leaderboard())


          rank  ensemble_weight               type      cost   duration
model_id                                                               
25           1             0.46                sgd  0.436679   0.740504
6            2             0.32     ard_regression  0.455042   0.839090
27           3             0.14     ard_regression  0.462249   0.732829
11           4             0.02      random_forest  0.507400  11.670796
7            5             0.06  gradient_boosting  0.518673   1.535592


In [7]:
pprint(automl.show_models(), indent=4)

{   6: {   'cost': 0.4550418898836528,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fdb3f7e7310>,
           'ensemble_weight': 0.32,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fdb3fa7e970>,
           'model_id': 6,
           'rank': 2,
           'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7fdb3fa7ecd0>,
           'sklearn_regressor': ARDRegression(alpha_1=0.0003701926442639788, alpha_2=2.2118001735899097e-07,
              copy_X=False, lambda_1=1.2037591637980971e-06,
              lambda_2=4.358378124977852e-09,
              threshold_lambda=1136.5286041327277, tol=0.021944240404849075)},
    7: {   'cost': 0.5186726734789994,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fdb3f7eb1c0>,
           'ensemble_

In [13]:
train_predictions = automl.predict(X_train)
print("Train R2 score:", sklearn.metrics.mean_squared_error(y_train, train_predictions, squared=False))
test_predictions = automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.mean_squared_error(y_test, test_predictions, squared=False))

Train R2 score: 49.97336721874017
Test R2 score: 56.15867945482282
